In [15]:
from feature_extractor import extract_string_lower
import os
from tqdm import tqdm
import pickle

In [16]:
from core.MV2 import MV2
from core.JIG import JIG
from core.SG2 import SG2

def GIPS(str_feature,
		 window_size, K, M, # MV2 파라미터
		 thetaJ,  # JIG 파라미터
		 vector_size, eps, minpts, ngram, hh1_size, hh2_size, hh3_size, ratio # SG2, AWL 파라미터
		):

	print(f'data no: {len(str_feature)}')

	str_feature = list(str_feature)
	for feature in str_feature:
		feature = list(feature)

	# 빅 그룹 식별
	minhashed_virtual_vectors = MV2(payloads=str_feature, window_size=window_size, K=K, M=M)

	big_group_indices = JIG(vectors=minhashed_virtual_vectors, thetaJ=thetaJ)
  
	
	big_group_payloads = []
	non_big_group_paylaods = []

	for idx, payload in enumerate(str_feature):
		if idx in big_group_indices:
			big_group_payloads.append(payload)
		else:
			non_big_group_paylaods.append(payload)
	
	# 시그니처 생성
	cluster_signatures = SG2(payloads=big_group_payloads, window_size=window_size, vector_size=vector_size, 
							 eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)

	return cluster_signatures

In [17]:
malware_direc = './datasets/PE_malware/'
benign_direc = './datasets/ben/'

In [18]:
malware_path = os.listdir(malware_direc)

with open('./datasets/pkl/benign_md5.pkl', 'rb') as f:
    benign_path = pickle.load(f)


malware_path.remove('README.md')

print(len(malware_path), len(benign_path))

29909 29909


In [19]:
benign_strings = [] # 전체 스트링 저장
white = [] # train 화이트 리스트
benign_test = [] # test benign

for path_ in tqdm(benign_path[: int(len(benign_path) * 0.8)]):
	path = os.path.join(benign_direc, path_)
	
	res = extract_string_lower(path=path)
	benign_strings.append(res)
	white.append(res)

KeyboardInterrupt: 

In [ ]:
white_list = set()

for inner in tqdm(white):
    white_list.update(inner)
        
print(len(white_list))

In [ ]:
malware_strings = [] # 전체 스트링 저장
malware_train = [] # train 화이트 리스트
malware_test = [] # test malware
benign_test = [] # test benign

for path_ in tqdm(malware_path[: int(len(malware_path) * 0.8)]):
	path = os.path.join(malware_direc, path_)
	
	res = extract_string_lower(path=path)
	malware_strings.append(res)

	tmp = []
	for string in res:
		if string not in white_list:
			tmp.append(string)

	malware_train.append(tmp)

for path_ in tqdm(malware_path[int(len(malware_path) * 0.8): ]):
	path = os.path.join(malware_direc, path_)
	
	res = extract_string_lower(path=path)
	malware_strings.append(res)

	tmp = []
	for string in res:
		if string not in white_list:
			tmp.append(string)

	malware_test.append(tmp)

for path_ in tqdm(benign_path[int(len(benign_path) * 0.8): ]):
	path = os.path.join(benign_direc, path_)
	
	res = extract_string_lower(path=path)
	benign_strings.append(res)

	tmp = []
	for string in res:
		if string not in white_list:
			tmp.append(string)

	benign_test.append(tmp)

In [ ]:
print(len(benign_strings), len(malware_strings))

with open('./datasets/pkl/low_benign_string.pkl', 'wb') as f:
    pickle.dump(benign_strings, f)
    
with open('./datasets/pkl/low_malware_string.pkl', 'wb') as f:
    pickle.dump(malware_strings, f)

In [ ]:
# parameters

K = 64
M = 2 ** 14
thetaJ = 0.6
window_size = 4
vector_size = 512
eps = 0.4
minpts = 5
ngram = 4
hh1_size = 5000
hh2_size = 5000
hh3_size = 5000
ratio = 0.8

In [ ]:
cluster_signatures = GIPS(str_feature=malware_train, window_size=window_size, K=K, M=M, thetaJ=thetaJ, vector_size=vector_size,
    eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio
)

In [ ]:
signatures = set()

for signature_list in cluster_signatures.values():
		for i in signature_list:
			signatures.add(i[0])

In [ ]:
print(len(signatures))

In [ ]:
for i in range(1, 20):
	res = []
	TP = 0
	FP = 0
	TN = 0
	FN = 0

	print('N', i)

	for strings in malware_test:
		cnt = 0
		for string in strings:
			if string in signatures:
				cnt += 1
		if cnt >= i:
			TP += 1
		else:
			FN += 1

	for strings in benign_test:
		cnt = 0
		for string in strings:
			if string in signatures:
				cnt += 1		
		if cnt >= i:
			FP += 1
		else:
			TN += 1
		
	# print(sum(benign) / len(benign))
	print(f'TP: {TP}, FN: {FN}')
	print(f'FP: {FP}, TN: {TN}')

	precision = round(TP / (TP + FP) , 4)
	recall = round(TP / (TP + FN), 4)

	print(f'precision: {precision}, recall: {recall}')
	print('f1-score:', round(2 * (precision * recall) / (precision + recall), 4))
	print()

# THH 크기 늘리기

In [ ]:
with open('./datasets/pkl/low_benign_string.pkl', 'rb') as f:
    benign_strings = pickle.load(f)
    
with open('./datasets/pkl/low_malware_string.pkl', 'rb') as f:
    malware_strings = pickle.load(f)

In [ ]:
import random

random.Random(42).shuffle(benign_strings)
random.Random(42).shuffle(malware_strings)

In [ ]:
white_list = dict()

for inner in tqdm(benign_strings[: int(len(benign_strings) * 0.8)]):
    for i in inner:
        if i not in white_list:
            white_list[i] = 0
        white_list[i] += 1
        
print(len(white_list))

In [ ]:
whitelist = set([i[0] for i in white_list.items() if i[1] > 0])
print(len(whitelist))

In [ ]:
malware_train = [] # train 화이트 리스트
malware_test = [] # test malware
benign_test = []

for res in tqdm(malware_strings[: int(len(malware_strings) * 0.8)]):
	
	tmp = []
	for string in res:
		if string not in whitelist:
			tmp.append(string)

	malware_train.append(tmp)

for res in tqdm(malware_strings[int(len(malware_strings) * 0.8): ]):

	tmp = []
	for string in res:
		if string not in whitelist:
			tmp.append(string)

	malware_test.append(tmp)

for res in tqdm(benign_strings[int(len(benign_strings) * 0.8): ]):

	tmp = []
	for string in res:
		if string not in whitelist:
			tmp.append(string)

	benign_test.append(tmp)

In [ ]:
# parameters

K = 64
M = 2 ** 14
thetaJ = 0.6
window_size = 4
vector_size = 512
eps = 0.4
minpts = 5
ngram = 4
hh1_size = 15000
hh2_size = 15000
hh3_size = 5000
ratio = 0.4

In [ ]:
cluster_signatures = GIPS(str_feature=malware_train, window_size=window_size, K=K, M=M, thetaJ=thetaJ, vector_size=vector_size,
    eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio
)

In [ ]:
signatures = set()

for signature_list in cluster_signatures.values():
		print(len(signature_list))
		for i in signature_list:
			signatures.add(i[0])

print(len(signatures))

In [ ]:
text = ''

for i in signatures:
    text += (i + '\n')
    
f = open('./signatures.txt', 'w', encoding='utf-8')
f.write(text)

In [ ]:
for i in range(1, 20):
	res = []
	TP = 0
	FP = 0
	TN = 0
	FN = 0

	print('N', i)

	for strings in malware_test:
		cnt = 0
		for string in strings:
			if string in signatures:
				cnt += 1
		if cnt >= i:
			TP += 1
		else:
			FN += 1

	for strings in benign_test:
		cnt = 0
		for string in strings:
			if string in signatures:
				cnt += 1		
		if cnt >= i:
			FP += 1
		else:
			TN += 1
		
	# print(sum(benign) / len(benign))
	print(f'TP: {TP}\tFN: {FN}')
	print(f'FP: {FP}\t\tTN: {TN}')

	precision = round(TP / (TP + FP) , 4)
	recall = round(TP / (TP + FN), 4)

	print(f'precision: {precision}, recall: {recall}')
	print('f1-score:', round(2 * (precision * recall) / (precision + recall), 4))
	print()

In [ ]:
dark_list = dict()

for inner in tqdm(malware_strings[: int(len(malware_strings) * 0.8)]):
    for i in inner:
        if i not in dark_list:
            dark_list[i] = 0
        dark_list[i] += 1
        
print(len(dark_list))

In [ ]:
cnt = 0

dark_lists = []

for i in sorted([i for i in dark_list.items()], key=lambda x: -x[1]):
	if i[0] not in white_list:
		dark_lists.append(i)

print(len(dark_lists))

In [ ]:
cnt = 0

nomi_sig = set()

threshold = (int(len(malware_strings) * 0.8) * 0.01)

for i in dark_lists:
    if i[1] > threshold:
        nomi_sig.add(i)

print(len(nomi_sig))
print(len(dark_lists) - len(nomi_sig))

In [ ]:
cnt_200s = set([i[0] for i in nomi_sig])

cnt = 0

for inner in tqdm(malware_strings[int(len(malware_strings) * 0.8): ]):
	for i in inner:
		if i in cnt_200s:
			cnt += 1
			break

print(cnt)